# Lab 3 – H2O AutoML (KNIME ↔ Python Companion Notebook)

This notebook mirrors the main blocks of the KNIME workflow in **`LAB3_AutoML.png`**:

1. **Data Preparation**: load CSV → rule-based target creation → one-hot encoding → feature selection → train/test split  
2. **Model Training**: H2O Local Context → Table to H2O → **H2O AutoML Learner**  
3. **Model Evaluation**: H2O Predictor → H2O to Table → Scorer  

> **Note:** Update the file path and the column names (`TARGET_COL`, categorical columns, etc.) to match your Lab 3 dataset.


In [24]:
# Run the below code if on COLAB
!pip -q install -U h2o

In [25]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import os
import h2o
from h2o.automl import H2OAutoML
import urllib.request

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [ ]:
DATA_URLS = {
    "Mall_Customers.csv": (
        "https://raw.githubusercontent.com/pvateekul/ieat2026/"
        "main/lab/LAB3_AutoML/Mall_Customers.csv"
    ),
}


def resolve_data_path(filename: str) -> str:
    candidate_dirs = [".", "/content"]

    # 1) Try local copies first
    for base in candidate_dirs:
        path = os.path.join(base, filename)
        if os.path.exists(path):
            print(f"Using local file: {os.path.abspath(path)}")
            return path

    # 2) Otherwise download from GitHub raw
    url = DATA_URLS[filename]
    target_path = os.path.join(os.getcwd(), filename)
    os.makedirs(os.path.dirname(target_path), exist_ok=True)

    print(f"Downloading '{filename}' from:\n  {url}\ninto:\n  {target_path}")
    urllib.request.urlretrieve(url, target_path)

    print(f"Downloaded: {os.path.abspath(target_path)}")
    return target_path

## 1) Data Preparation (mirrors CSV Reader → Rule Engine → One to Many → Column Filter → Table Partitioner)

### 1.1 Load CSV (CSV Reader)
Set `CSV_PATH` to the same dataset you used in KNIME.


In [27]:
df = pd.read_csv(resolve_data_path("Mall_Customers.csv"))
df.head()

Using local file: /Users/theerapatkangsilalai/Documents/GitHub/ieat2026/lab/LAB3_AutoML/Mall_Customers.csv


,CustomerID,Gender,Age,Annual Income (k$),Spending Score (1-100)
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40


### 1.2 Create / clean the class label (Rule Engine)

In KNIME, the **Rule Engine** is often used to:
- map numeric codes to labels (e.g., 0/1 → "No"/"Yes")
- create a label based on thresholds / business rules

Edit this section to match your dataset.


In [28]:
RAW_TARGET_COL = "Spending Score (1-100)"
TARGET_COL = "High Spender"

if RAW_TARGET_COL not in df.columns:
    raise ValueError(
        f"Column not found: {RAW_TARGET_COL}. Available: {list(df.columns)}")

df[TARGET_COL] = (df[RAW_TARGET_COL] >= 60).astype(int)

df[TARGET_COL].value_counts()

High Spender
0    135
1     65
Name: count, dtype: int64

### 1.3 One-hot encode categorical predictors (One to Many)

H2O can natively handle categoricals (as factors), but one-hot encoding is a common explicit preprocessing step.
If you want to keep categoricals as factors (recommended for H2O), you can skip `get_dummies` and instead convert them to `asfactor()` after importing into H2O.


In [29]:
DROP_COLS = ["CustomerID", "Spending Score (1-100)"]

df_model = df.drop(columns=[c for c in DROP_COLS if c in df.columns]).copy()

cat_cols = [c for c in df_model.columns if df_model[c].dtype == "object" and c != TARGET_COL]
df_model = pd.get_dummies(df_model, columns=cat_cols, drop_first=True)

df_model.shape, df_model.columns[:10]

((200, 4),
 Index(['Age', 'Annual Income (k$)', 'High Spender', 'Gender_Male'], dtype='object'))

### 1.4 Train/Test split (Table Partitioner)

Use stratification for classification (especially if classes are imbalanced), and fix the random seed for reproducibility.


In [30]:
SEED = 2026
test_size = 0.20

X = df_model.drop(columns=[TARGET_COL])
y = df_model[TARGET_COL].astype(str)

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=test_size,
    random_state=SEED,
    stratify=y
)

X_train.shape, X_test.shape, y_train.value_counts(normalize=True)


((160, 3),
 (40, 3),
 High Spender
 0    0.675
 1    0.325
 Name: proportion, dtype: float64)

## 2) Model Training (mirrors H2O Local Context → Table to H2O → H2O AutoML Learner)

### 2.1 Start H2O (H2O Local Context)


In [31]:
h2o.init(max_mem_size="4G")  
h2o.no_progress()  


Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,3 hours 14 mins
H2O_cluster_timezone:,Asia/Bangkok
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.9
H2O_cluster_version_age:,2 months and 25 days
H2O_cluster_name:,H2O_from_python_theerapatkangsilalai_dier9b
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.217 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


### 2.2 Convert to H2OFrames (Table to H2O)

We join `X` and `y` so H2O has the response column in the same frame.


In [32]:
train_h2o = h2o.H2OFrame(pd.concat([X_train, y_train.rename(TARGET_COL)], axis=1))
test_h2o  = h2o.H2OFrame(pd.concat([X_test,  y_test.rename(TARGET_COL)], axis=1))

train_h2o[TARGET_COL] = train_h2o[TARGET_COL].asfactor()
test_h2o[TARGET_COL]  = test_h2o[TARGET_COL].asfactor()

x_cols = [c for c in train_h2o.columns if c != TARGET_COL]
x_cols[:5], TARGET_COL


(['Age', 'Annual Income (k$)', 'Gender_Male'], 'High Spender')

### 2.3 Train H2O AutoML (H2O AutoML Learner)

Control training time using `max_models` or `max_runtime_secs`.


In [33]:
aml = H2OAutoML(
    max_models=20,
    seed=SEED,
    sort_metric="AUTO"
)

aml.train(x=x_cols, y=TARGET_COL, training_frame=train_h2o)

lb = aml.leaderboard
lb.head(rows=10)



19:37:29.443: AutoML: XGBoost is not available; skipping it.


19:37:29.777: _min_rows param, The dataset size is too small to split for min_rows=100.0: must have at least 200.0 (weighted) rows, but have only 160.0.



model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
GBM_grid_1_AutoML_2_20260218_193729_model_2,0.920139,0.327829,0.84109,0.117165,0.320918,0.102988
GBM_4_AutoML_2_20260218_193729,0.918625,0.332879,0.795267,0.117165,0.322787,0.104192
GBM_grid_1_AutoML_2_20260218_193729_model_5,0.914174,0.35381,0.811581,0.135684,0.337686,0.114032
StackedEnsemble_BestOfFamily_1_AutoML_2_20260218_193729,0.914174,0.342576,0.791805,0.131054,0.32948,0.108557
GBM_5_AutoML_2_20260218_193729,0.911058,0.35372,0.798678,0.134972,0.338887,0.114845
GBM_3_AutoML_2_20260218_193729,0.910078,0.346389,0.798391,0.126781,0.333262,0.111064
GBM_grid_1_AutoML_2_20260218_193729_model_1,0.909811,0.34663,0.827424,0.126068,0.328754,0.108079
StackedEnsemble_AllModels_1_AutoML_2_20260218_193729,0.907585,0.356799,0.78166,0.149929,0.338691,0.114712
XRT_1_AutoML_2_20260218_193729,0.895299,0.398902,0.74503,0.116453,0.354805,0.125887
GBM_2_AutoML_2_20260218_193729,0.894587,0.358244,0.779526,0.126425,0.33687,0.113482


## 3) Model Evaluation (mirrors H2O Predictor → H2O to Table → Scorer)

In [34]:
pred_h2o = aml.leader.predict(test_h2o)

pred_df = pred_h2o.as_data_frame()
pred_df.head()


/opt/miniconda3/envs/dsde/lib/python3.14/site-packages/h2o/frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


,predict,p0,p1
0,0,0.979604,0.020396
1,0,0.983861,0.016139
2,1,0.065302,0.934698
3,0,0.802233,0.197767
4,0,0.980875,0.019125


In [35]:
y_pred = pred_df["predict"].astype(str).to_numpy()
y_true = y_test.astype(str).to_numpy()

print("Accuracy:", accuracy_score(y_true, y_pred))
print("\nConfusion matrix:")
print(confusion_matrix(y_true, y_pred))
print("\nClassification report:")
print(classification_report(y_true, y_pred, digits=4))


Accuracy: 0.725

Confusion matrix:
[[20  7]
 [ 4  9]]

Classification report:
              precision    recall  f1-score   support

           0     0.8333    0.7407    0.7843        27
           1     0.5625    0.6923    0.6207        13

    accuracy                         0.7250        40
   macro avg     0.6979    0.7165    0.7025        40
weighted avg     0.7453    0.7250    0.7311        40

